# Tugas Pertemuan 53
| Label       | Informasi                        |
|-------------|----------------------------------|
| Nama        | Khoirul Huda                     |
| NIM         | 22.230.0118                      |
| Universitas | Institut Widya Pratama           |
| Kelas       | Bravo (Malam)                    |

In [1]:
pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk
import mysql.connector

# Koneksi ke database MySQL
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="",  # Masukkan password MySQL Anda
        database="school"
    )

# Fungsi untuk menambahkan data
def add_data():
    name = name_entry.get()
    age = age_entry.get()
    grade = grade_entry.get()
    if name and age and grade:
        try:
            conn = connect_to_db()
            cursor = conn.cursor()
            cursor.execute("INSERT INTO students (name, age, grade) VALUES (%s, %s, %s)", (name, age, grade))
            conn.commit()
            conn.close()
            fetch_data()
            clear_fields()
            messagebox.showinfo("Success", "Data added successfully!")
        except Exception as e:
            messagebox.showerror("Error", str(e))
    else:
        messagebox.showwarning("Input Error", "All fields are required!")

# Fungsi untuk mengambil data
def fetch_data():
    try:
        conn = connect_to_db()
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM students")
        rows = cursor.fetchall()
        table.delete(*table.get_children())
        for row in rows:
            table.insert("", tk.END, values=row)
        conn.close()
    except Exception as e:
        messagebox.showerror("Error", str(e))

# Fungsi untuk memperbarui data
def update_data():
    selected_item = table.selection()
    if selected_item:
        item = table.item(selected_item)
        student_id = item['values'][0]
        name = name_entry.get()
        age = age_entry.get()
        grade = grade_entry.get()
        if name and age and grade:
            try:
                conn = connect_to_db()
                cursor = conn.cursor()
                cursor.execute(
                    "UPDATE students SET name=%s, age=%s, grade=%s WHERE id=%s",
                    (name, age, grade, student_id)
                )
                conn.commit()
                conn.close()
                fetch_data()
                clear_fields()
                messagebox.showinfo("Success", "Data updated successfully!")
            except Exception as e:
                messagebox.showerror("Error", str(e))
        else:
            messagebox.showwarning("Input Error", "All fields are required!")
    else:
        messagebox.showwarning("Selection Error", "No item selected!")

# Fungsi untuk menghapus data
def delete_data():
    selected_item = table.selection()
    if selected_item:
        item = table.item(selected_item)
        student_id = item['values'][0]
        try:
            conn = connect_to_db()
            cursor = conn.cursor()
            cursor.execute("DELETE FROM students WHERE id=%s", (student_id,))
            conn.commit()
            conn.close()
            fetch_data()
            clear_fields()
            messagebox.showinfo("Success", "Data deleted successfully!")
        except Exception as e:
            messagebox.showerror("Error", str(e))
    else:
        messagebox.showwarning("Selection Error", "No item selected!")

# Fungsi untuk membersihkan input field
def clear_fields():
    name_entry.delete(0, tk.END)
    age_entry.delete(0, tk.END)
    grade_entry.delete(0, tk.END)

# Fungsi untuk menampilkan data ke field input
def fill_fields(event):
    selected_item = table.selection()
    if selected_item:
        item = table.item(selected_item)
        values = item['values']
        name_entry.delete(0, tk.END)
        name_entry.insert(0, values[1])
        age_entry.delete(0, tk.END)
        age_entry.insert(0, values[2])
        grade_entry.delete(0, tk.END)
        grade_entry.insert(0, values[3])

# GUI Tkinter
root = tk.Tk()
root.title("CRUD Application with Tkinter and MySQL")
root.geometry("600x400")

# Frame Input
input_frame = tk.Frame(root)
input_frame.pack(pady=10)

tk.Label(input_frame, text="Name:").grid(row=0, column=0, padx=5, pady=5)
name_entry = tk.Entry(input_frame)
name_entry.grid(row=0, column=1, padx=5, pady=5)

tk.Label(input_frame, text="Age:").grid(row=1, column=0, padx=5, pady=5)
age_entry = tk.Entry(input_frame)
age_entry.grid(row=1, column=1, padx=5, pady=5)

tk.Label(input_frame, text="Grade:").grid(row=2, column=0, padx=5, pady=5)
grade_entry = tk.Entry(input_frame)
grade_entry.grid(row=2, column=1, padx=5, pady=5)

# Buttons
button_frame = tk.Frame(root)
button_frame.pack(pady=10)

tk.Button(button_frame, text="Add", command=add_data).grid(row=0, column=0, padx=5)
tk.Button(button_frame, text="Update", command=update_data).grid(row=0, column=1, padx=5)
tk.Button(button_frame, text="Delete", command=delete_data).grid(row=0, column=2, padx=5)
tk.Button(button_frame, text="Clear", command=clear_fields).grid(row=0, column=3, padx=5)

# Table
table_frame = tk.Frame(root)
table_frame.pack(pady=10)

table = ttk.Treeview(table_frame, columns=("ID", "Name", "Age", "Grade"), show="headings")
table.heading("ID", text="ID")
table.heading("Name", text="Name")
table.heading("Age", text="Age")
table.heading("Grade", text="Grade")
table.pack()

table.bind("<ButtonRelease-1>", fill_fields)

# Fetch data on start
fetch_data()

# Main loop
root.mainloop()